<a href="https://colab.research.google.com/github/Malik-Raheel/Trading-Strategies/blob/main/Preditive_Analysis_intraday_trading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trading PRedictive Models for Predictive values

In [1]:
pip install ccxt pandas numpy ta scikit-learn matplotlib seaborn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.6/288.6 kB 15.9 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=29ca1ce5bcce1a39a9a2ffb8a292c0192fe8f2435276dc758b56bb4e87abf6cf
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta
  Attempting uninstall: aiohttp
    Found existing installation: aiohttp 3.11.11
    Uninstalling aiohttp-3.11.11:
      Successfully uninstalled aiohttp-3.11.11


In [4]:
# Required Libraries
import ccxt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from ta.momentum import RSIIndicator
from ta.volatility import AverageTrueRange
from ta.volume import OnBalanceVolumeIndicator
from ta.trend import MACD

# Initialize the Exchange
exchange = ccxt.kucoin()  # Replace 'kucoin' with your preferred exchange

# List of Crypto Symbols
symbols = [
    "BTC/USDT", "ETH/USDT", "AVAX/USDT", "SOL/USDT", "XAVA/USDT",
    "SLIM/USDT", "HNT/USDT", "NEAR/USDT", "DOGE/USDT", "SAND/USDT",
    "CRV/USDT", "CFX/USDT", "INJ/USDT", "LINK/USDT", "AAVE/USDT",
    "LTC/USDT", "ADA/USDT", "GAFI/USDT", "JUP/USDT", "MNT/USDT",
    "PYTH/USDT", "STG/USDT"
]

# Function to Fetch Historical Data
def fetch_ccxt_data(symbol, timeframe, limit=500):
    """Fetch historical OHLCV data using CCXT."""
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
    df = pd.DataFrame(ohlcv, columns=["time", "open", "high", "low", "close", "volume"])
    df["time"] = pd.to_datetime(df["time"], unit="ms")
    return df

# Function to Add Indicators
def calculate_indicators(df):
    """Calculate RSI, ATR, MACD, and OBV indicators for the dataframe."""
    df["RSI"] = RSIIndicator(df["close"], window=14).rsi()
    atr = AverageTrueRange(df["high"], df["low"], df["close"], window=14)
    df["ATR"] = atr.average_true_range()
    macd = MACD(df["close"])
    df["MACD"] = macd.macd()
    df["Signal"] = macd.macd_signal()
    df["OBV"] = OnBalanceVolumeIndicator(df["close"], df["volume"]).on_balance_volume()
    return df

# Function to Filter Assets
def filter_assets(data):
    """Filter assets based on RSI, MACD, ATR, and OBV criteria."""
    return (
        data["RSI"].iloc[-1] < 30  # RSI in oversold zone
        and data["MACD"].iloc[-1] > data["Signal"].iloc[-1]  # Bullish MACD crossover
        and data["ATR"].iloc[-1] > 0  # Positive ATR
        and data["OBV"].iloc[-1] > 0  # Positive OBV
    )

# Function to Train Regression Models
def train_regression_models(df):
    """Train a regression model to predict future price changes."""
    df["returns"] = df["close"].pct_change()  # Percentage change in closing price
    df = df.dropna()

    # Prepare Features and Target
    X = df[["RSI", "ATR", "MACD"]].values
    y = df["returns"].shift(-1).dropna().values
    X = X[:-1]

    # Scale Features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Train Regression Model
    model = LinearRegression()
    model.fit(X_scaled, y)

    return model, scaler

# Function to Predict Future Prices
def predict_future(model, scaler, current_features):
    """Predict future returns using the trained model."""
    X_input = scaler.transform([current_features])
    return model.predict(X_input)[0]

# Main Execution
filtered_assets = []
regression_models = {}

for symbol in symbols:
    try:
        print(f"Processing symbol: {symbol}")

        # Fetch Historical Data
        data_4h = fetch_ccxt_data(symbol, "4h")
        data_4h = calculate_indicators(data_4h)

        # Apply Filtering Criteria
        if filter_assets(data_4h):
            filtered_assets.append(symbol)

            # Train Regression Model
            model, scaler = train_regression_models(data_4h)
            regression_models[symbol] = (model, scaler)

            # Predict Next 4-Hour Return
            current_features = [
                data_4h["RSI"].iloc[-1],
                data_4h["ATR"].iloc[-1],
                data_4h["MACD"].iloc[-1]
            ]
            predicted_return = predict_future(model, scaler, current_features)

            print(f"{symbol}: Predicted 4-hour return: {predicted_return:.2%}")

    except Exception as e:
        print(f"Error processing {symbol}: {e}")

# Display Filtered Assets
if filtered_assets:
    print("\nFiltered Assets for Today's Trade:")
    for asset in filtered_assets:
        print(asset)
else:
    print("\nNo assets met the filtering criteria today.")


Processing symbol: BTC/USDT
Processing symbol: ETH/USDT
Processing symbol: AVAX/USDT
Processing symbol: SOL/USDT
Processing symbol: XAVA/USDT
Processing symbol: SLIM/USDT
Processing symbol: HNT/USDT
Processing symbol: NEAR/USDT
Processing symbol: DOGE/USDT
Processing symbol: SAND/USDT
Processing symbol: CRV/USDT
Processing symbol: CFX/USDT
Processing symbol: INJ/USDT
Processing symbol: LINK/USDT
Processing symbol: AAVE/USDT
Processing symbol: LTC/USDT
Processing symbol: ADA/USDT
Processing symbol: GAFI/USDT
Processing symbol: JUP/USDT
Processing symbol: MNT/USDT
Processing symbol: PYTH/USDT
Processing symbol: STG/USDT

No assets met the filtering criteria today.
